<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Before-you-start" data-toc-modified-id="Before-you-start-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Before you start</a></span></li><li><span><a href="#Set-Up-Authentication" data-toc-modified-id="Set-Up-Authentication-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set Up Authentication</a></span></li><li><span><a href="#Build-the-EOSS-Root-URL" data-toc-modified-id="Build-the-EOSS-Root-URL-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Build the EOSS Root URL</a></span></li><li><span><a href="#Variable-Subset-of-a-Granule" data-toc-modified-id="Variable-Subset-of-a-Granule-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Variable Subset of a Granule</a></span></li><li><span><a href="#Add-on-a-spatial-subset" data-toc-modified-id="Add-on-a-spatial-subset-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Add on a spatial subset</a></span></li><li><span><a href="#Reprojection" data-toc-modified-id="Reprojection-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Reprojection</a></span></li><li><span><a href="#Reformatting" data-toc-modified-id="Reformatting-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Reformatting</a></span></li><li><span><a href="#Continue-Exploring" data-toc-modified-id="Continue-Exploring-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Continue Exploring</a></span></li></ul></div>

# Harmony EOSS API Tutorial

## Before you start
Before you beginning this tutorial, make sure you have an account in the Earthdata Login UAT environment, which 
will be used for this notebook by visiting [https://uat.urs.earthdata.nasa.gov](https://uat.urs.earthdata.nasa.gov).
These accounts, as all Earthdata Login accounts, are free to create and only take a moment to set up.

## Set Up Authentication

We need some boilerplate up front to log in to Earthdata Login.  The function below will allow Python
scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine uat.urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 


In [ ]:
from urllib import request
from http.cookiejar import CookieJar
import getpass
import netrc

def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)


Now call the above function to set up Earthdata Login for subsequent requests

In [ ]:
setup_earthdata_login_auth('uat.urs.earthdata.nasa.gov')

## Build the EOSS Root URL

Next we will build a URL for the EOSS service for a given granule.  To get data using the service, you need 
a [CMR UAT](https://cmr.uat.earthdata.nasa.gov) collection ID for a supported collection and the ID of a
granule within that collection.

By convention, all Harmony services are accessed through `<harmony_root>/<collection_id>/<service_name>`

In [ ]:
harmony_root = 'https://harmony.uat.earthdata.nasa.gov'
config = {
    'collection_id': 'C1233800302-EEDTEST',
    'eoss_version': '0.1.0'
}
eoss_collection_root = harmony_root+'/{collection_id}/eoss/{eoss_version}/items/'.format(**config)
print(eoss_collection_root)

## Variable Subset of a Granule

We can now build onto the root URL in order to actually perform a transformation.  The first transformation is a variable subset of a selected granule.  _At this time, this requires discovering the granule id and variable id from CMR_.  That information can then be appended to the root URL and used to call Harmony with the help of the `request` library.

Harmony stages transformed data in S3 to make it easy to do additional processing in the cloud. The response that Harmony returns is actually a redirect to the S3 location where your data is staged.  Should you call Harmony in a tool that follows redirects, like your web browser, your file will be seamlessly downloaded locally for you.  However, should you desire to do additional processing in AWS, you have that option as well by simply looking at the redirected URL.  The code snippet below uses "geturl()" to show the URL of your staged data.

In [ ]:
varSubsetConfig = {
    'granule_id' : 'G1233800343-EEDTEST',
    'variable_id' : 'red_var'
}
eoss_var_subset_url = eoss_collection_root+'{granule_id}/?rangeSubset={variable_id}'.format(**varSubsetConfig)

print('Request URL', eoss_var_subset_url)

with request.urlopen(eoss_var_subset_url) as response:
    print('URL for data staged in S3:', response.geturl())

## Add on a spatial subset

The second transformation is a spatial subset of a selected granule. This can be combined with the request we already built above by simply specifying a bounding box.

In [ ]:
spatialSubsetConfig = {
    'west' : '-128',
    'south' : '23',
    'east' : '-63',
    'north' : '47'
}
eoss_spatial_subset_url = eoss_var_subset_url+'&bbox={west},{south},{east},{north}'.format(**spatialSubsetConfig)

print('Request URL', eoss_spatial_subset_url)

with request.urlopen(eoss_spatial_subset_url) as response:
    print('URL for data staged in S3:', response.geturl())

## Reprojection

The third transformation is a reprojection of the data. This can be combined with the requests we already built above by simply specifying a coordinate reference system.  Coordinate reference systems are identified by a common name, EPSG code, or URI. Today, this is based on reference systems supported by gdal. Examples include: 'CRS:84', 'EPSG:32611'.

In [ ]:
reprojectionConfig = {
    'crs' : 'EPSG:32611'
}
eoss_reprojection_url = eoss_spatial_subset_url+'&crs={crs}'.format(**reprojectionConfig)

print('Request URL', eoss_reprojection_url)

with request.urlopen(eoss_reprojection_url) as response:
    print('URL for data staged in S3:', response.geturl())


## Reformatting

Next is a reformatting of the output file of the data. This can be combined with the requests we already built above by simply specifying a format.  Examples include: image/tiff', 'image/png'

In [ ]:
reformattingConfig = {
    'format' : 'image/png'
}
eoss_reformatting_url = eoss_reprojection_url+'&format={format}'.format(**reformattingConfig)

print('Request URL', eoss_reformatting_url)

with request.urlopen(eoss_reformatting_url) as response:
    print('URL for data staged in S3:', response.geturl())


## Continue Exploring

Harmony's specification is available online.  Feel free to read more and continue exploring how to use Harmony. https://harmony.uat.earthdata.nasa.gov/docs/eoss/0.1.0/spec